# **GRADIENT DECENT**

In [10]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
%matplotlib inline

In [12]:
df = pd.read_csv('S:/Github/DATASET/insurance_data.csv')
df.head(5)

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [60]:
x_train, x_test, y_train, y_test = train_test_split(
    pd.concat([df['age'] / 100, df['affordibility']], axis=1),
    df['bought_insurance'],
    test_size=0.2,
    random_state=42
)

x_train.shape , x_test.shape , y_train.shape , y_test.shape


((22, 2), (6, 2), (22,), (6,))

In [44]:
model = keras.Sequential([
    keras.layers.Dense(1,activation='sigmoid',input_shape=(2,),kernel_initializer='ones',bias_initializer='zeros'),

])

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.summary()
model.fit(x_train,y_train,epochs=5000,validation_split=0.2)

c:\Users\13zer\anaconda3\envs\dl\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_5 (Dense)                 │ (None, 1)              │             3 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3 (12.00 B)

 Trainable params: 3 (12.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.5294 - loss: 0.6941 - val_accuracy: 0.4000 - val_loss: 0.9078
Epoch 2/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step - accuracy: 0.5294 - loss: 0.6938 - val_accuracy: 0.4000 - val_loss: 0.9071
Epoch 3/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step - accuracy: 0.5294 - loss: 0.6934 - val_accuracy: 0.4000 - val_loss: 0.9065
Epoch 4/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step - accuracy: 0.5294 - loss: 0.6931 - val_accuracy: 0.4000 - val_loss: 0.9058
Epoch 5/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step - accuracy: 0.5294 - loss: 0.6927 - val_accuracy: 0.4000 - val_loss: 0.9051
Epoch 6/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step - accuracy: 0.5294 - loss: 0.6924 - val_accuracy: 0.4000 - val_loss: 0.9045
Epoch 7/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step - accuracy: 0.5294 - loss: 0.6920 - val_accuracy: 0.4000 - val_loss: 0.9038
Epoch 8/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step - accuracy: 0.5294 - loss: 0.6917 - val_accuracy: 0.400

In [45]:
coef,intercept = model.layers[0].get_weights()
print(coef,intercept)

[[4.8386726]
 [1.4275059]] [-2.7634237]


In [46]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [55]:
def predicate(age, affordibility):
    x = np.array([[age / 100, affordibility]])
    prefiction = coef[0]*x[0][0] + coef[1]*x[0][1] + intercept
    return sigmoid(prefiction)[0]


prediction = predicate(47, 1)
print(f'Prediction: {prediction}')

Prediction: 0.7187477098084625


In [68]:
def log_loss(y_true, y_predicted):
    epsilon = 1e-15
    y_predicted_new = [max(i,epsilon) for i in y_predicted]
    y_predicted_new = [min(i,1-epsilon) for i in y_predicted_new]
    y_predicted_new = np.array(y_predicted_new)
    return -np.mean(y_true*np.log(y_predicted_new)+(1-y_true)*np.log(1-y_predicted_new))

In [69]:
def gradient_descent(age, affordability, y_true, epochs, loss_thresold):
    w1 = w2 = 1
    bias = 0
    rate = 0.5
    n = len(age)
    for i in range(epochs):
        weighted_sum = w1 * age + w2 * affordability + bias
        y_predicted = sigmoid_numpy(weighted_sum)
        loss = log_loss(y_true, y_predicted)

        w1d = (1/n)*np.dot(np.transpose(age),(y_predicted-y_true)) 
        w2d = (1/n)*np.dot(np.transpose(affordability),(y_predicted-y_true)) 

        bias_d = np.mean(y_predicted-y_true)
        w1 = w1 - rate * w1d
        w2 = w2 - rate * w2d
        bias = bias - rate * bias_d

        print (f'Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')

        if loss<=loss_thresold:
            break

    return w1, w2, bias

In [70]:
gradient_descent(x_train['age']/100,x_train['affordibility'],y_train,5000, 0.4631)

Epoch:0, w1:0.9999069139682365, w2:0.9550531097670333, bias:-0.07918836722847843, loss:0.7070459198423374
Epoch:1, w1:0.9998598981254031, w2:0.9186460534643577, bias:-0.1466887714351632, loss:0.6917636904825727
Epoch:2, w1:0.9998529902651214, w2:0.8897342474109293, bias:-0.20401747735995687, loss:0.6809544326850906
Epoch:3, w1:0.9998804251448297, w2:0.867267071543046, bias:-0.25263828319495396, loss:0.6733923290900665
Epoch:4, w1:0.9999369005608673, w2:0.8502535093530555, bias:-0.2938959607064187, loss:0.6681339751071751
Epoch:5, w1:1.0000177080012764, w2:0.8377983456747331, bias:-0.32898380708389685, loss:0.6644820825734673
Epoch:6, w1:1.0001187682060753, w2:0.8291167561973837, bias:-0.35893509342208296, loss:0.6619363932643565
Epoch:7, w1:1.0002366084459993, w2:0.8235350242484127, bias:-0.384629121313295, loss:0.6601457367681364
Epoch:8, w1:1.0003683089847188, w2:0.8204834084589505, bias:-0.4068049726593189, loss:0.6588674751546629
Epoch:9, w1:1.0005114368072894, w2:0.819485263512214

(np.float64(2.0775245402498297),
 np.float64(1.3212905548448763),
 np.float64(-0.9241341551346955))